In [1]:
import os; os.chdir('..')

In [2]:
# actual query logs present in google along with their frequency of search
demo_query_logs= {
    'what is translation earbuds?': 3,
    'how translation earbuds work?': 10, # most frequent
    'what are the best translation earbuds?': 1,
    'what is the most expensive translation earbud?':0
}

In [3]:
query_templates= ['define {}', 'what do you mean by {}', 'what are the best {}', 'how heavy are {}', 'what is color of {}', 'cheapest {}', 'working of {}']

In [4]:
query= "translation earbuds"

In [5]:
# template acquisition

acquired_templates= []
for i in query_templates:
    acquired_templates.append(i.format(query))
    
    
acquired_templates

['define translation earbuds',
 'what do you mean by translation earbuds',
 'what are the best translation earbuds',
 'how heavy are translation earbuds',
 'what is color of translation earbuds',
 'cheapest translation earbuds',
 'working of translation earbuds']

In [6]:
from main import generate_cosine_similarity, generate_base_embeddings_v2

In [7]:
acquired_templates_embeddings= generate_base_embeddings_v2(acquired_templates)
acquired_templates_embeddings.shape

(7, 1024)

In [11]:
query_logs= list(demo_query_logs.keys())
query_logs_embeddings= generate_base_embeddings_v2(query_logs)
query_logs_embeddings.shape

(4, 1024)

In [19]:
similarity_matrix= generate_cosine_similarity(acquired_templates_embeddings, query_logs_embeddings).tolist()
similarity_matrix

[[0.9890637993812561,
  0.9642318487167358,
  0.9445757865905762,
  0.9199326038360596],
 [0.9841970205307007,
  0.9717227816581726,
  0.9531764984130859,
  0.9242734909057617],
 [0.9511412978172302,
  0.9291219115257263,
  0.9983932375907898,
  0.944415807723999],
 [0.9125149250030518,
  0.8945550918579102,
  0.9029437899589539,
  0.8925219774246216],
 [0.9255924820899963,
  0.9049127101898193,
  0.9084018468856812,
  0.8920938968658447],
 [0.9308352470397949,
  0.9122565388679504,
  0.948818564414978,
  0.9401549696922302],
 [0.962012529373169,
  0.9773651957511902,
  0.9406726360321045,
  0.9189484119415283]]

In [30]:
# semantically similar keywords
semantically_similar_query= []
for i in range(len(similarity_matrix)):
    max_sim= max(similarity_matrix[i])
    original_query= query_logs[similarity_matrix[i].index(max_sim)]
    if max_sim>0.95:
        semantically_similar_query.append((acquired_templates[i], original_query, demo_query_logs[original_query]))
# semantically_similar_query

In [31]:
for i in semantically_similar_query:
    print(f'query: "{i[0]}" \t\t\toriginal_query: "{i[1]}"')

query: "define translation earbuds" 			original_query: "what is translation earbuds?"
query: "what do you mean by translation earbuds" 			original_query: "what is translation earbuds?"
query: "what are the best translation earbuds" 			original_query: "what are the best translation earbuds?"
query: "working of translation earbuds" 			original_query: "how translation earbuds work?"


In [35]:
# Re Ranking based on #number of  searches
semantically_similar_query.sort(key=lambda x:x[2], reverse=True)

In [37]:
for i in semantically_similar_query:
    print(f"generated query: '''{i[0]}''', original query: '''{i[1]}''', #times: {i[2]}")

generated query: '''working of translation earbuds''', original query: '''how translation earbuds work?''', #times: 10
generated query: '''define translation earbuds''', original query: '''what is translation earbuds?''', #times: 3
generated query: '''what do you mean by translation earbuds''', original query: '''what is translation earbuds?''', #times: 3
generated query: '''what are the best translation earbuds''', original query: '''what are the best translation earbuds?''', #times: 1
